<a href="https://colab.research.google.com/github/ntq2503/Anti-Spam-ML-Project/blob/master/Anti_Spam_Quan_Nguyen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving spam.csv to spam.csv
Saving spam_utils.py to spam_utils.py
User uploaded file "spam.csv" with length 503663 bytes
User uploaded file "spam_utils.py" with length 2822 bytes


In [67]:
import numpy as np
import pandas as pd
from spam_utils import *
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Download Global Vectors for word representations 

!wget "https://drive.google.com/uc?id=10T_GDYRmM8OJ8TA9hdFrXM1HQXvTEK4p&export=download" -O glove.6B.50d.txt.zip
!unzip glove.6B.50d.txt.zip
!rm glove.6B.50d.txt.zip
#!mv glove.6B.50d.txt data/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
--2019-02-18 00:18:39--  https://drive.google.com/uc?id=10T_GDYRmM8OJ8TA9hdFrXM1HQXvTEK4p&export=download
Resolving drive.google.com (drive.google.com)... 74.125.204.138, 74.125.204.101, 74.125.204.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.204.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g54tu9d01ekopfa8tp4qv8m6dtdbdctq/1550448000000/01242245054912888515/*/10T_GDYRmM8OJ8TA9hdFrXM1HQXvTEK4p?e=download [following]
--2019-02-18 00:18:40--  https://doc-0c-74-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/g54tu9d01ekopfa8tp4qv8m6dtdbdctq/1550448000000/01242245054912888515/*/10T_GDYRmM8OJ8TA9hdFrXM1HQXvTEK4p?e=download
Resolving doc-0c-74-docs.googleusercontent.com (doc-0c-74-docs.goog

In [102]:
# create an object representing data set 
sms = pd.read_csv("spam.csv", usecols=[0,1], names=["label", "message"], skiprows=1)
sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [103]:
# map spam to 1 and non-spam to 0
sms.label = sms.label.map({"ham":0, "spam":1})
sms.label.value_counts()

0    4825
1     747
Name: label, dtype: int64

In [0]:
word_to_index, index_to_word, word_to_vec_map, words = read_glove_vecs('glove.6B.50d.txt')

In [0]:
# function to process text messages, removing punctuations and stopwords
import string
from string import maketrans

def text_process(text):    
    text.translate(maketrans(",", " "))
    text.translate(maketrans(".", " "))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    text = [word for word in text if word.lower() in words]
    return " ".join(text)


In [106]:
sms.message = sms.message.apply(text_process)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  import sys


In [107]:
sms.head()

,label,message
0,0,Go jurong Available bugis n great world la e C...
1,0,Ok Joking wif u
2,1,Free entry 2 comp win FA Cup final tkts 21st M...
3,0,U dun say early U c already
4,0,Nah think goes lives around though


In [0]:
# turn data set into NumPy arrays

x = np.array(sms.message.tolist())
y = np.array(sms.label.tolist())

In [0]:
# split data into training and testing set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [0]:
# function to turn a sentence into a vector encoding the meaning

def sentence_to_avg(sentence, word_to_vec_map):
   word = sentence.lower().split()
   avg = np.zeros((50,))
   
   for w in word:     
        avg += word_to_vec_map[w]     
        
   avg = avg/ len(word)
   return avg

In [111]:
avg = sentence_to_avg("hello world", word_to_vec_map)
print("avg = ", avg)

('avg = ', array([-3.999150e-01,  7.597000e-01, -1.201970e-01,  2.954500e-01,
        9.882550e-02, -4.551000e-01, -5.090950e-01, -2.826500e-01,
        1.404500e-01,  2.194200e-01,  3.288735e-01,  3.987600e-01,
       -3.585900e-01,  5.381550e-01,  1.024800e+00, -1.412195e-01,
       -6.614000e-02,  5.867300e-01, -5.714700e-01, -1.098525e-01,
       -4.170850e-02,  2.911685e-01,  9.799500e-02,  3.034850e-01,
        8.380850e-01, -1.147305e+00, -8.838050e-01, -2.613000e-02,
        4.856500e-02, -3.882150e-01,  2.053510e+00,  6.011650e-01,
       -2.663175e-01,  3.435400e-01, -9.298150e-01, -2.343700e-01,
        1.258950e-01, -1.491650e-01, -4.573615e-01, -6.752050e-01,
       -4.088000e-02, -5.184800e-01,  2.271900e-01, -4.466850e-01,
        5.242400e-01,  4.384400e-01, -7.453550e-02, -5.183800e-01,
       -1.637785e-01, -1.690000e-03]))


In [0]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
  
  m = Y.shape[0]          #number of training examples        
  n_y = 2                 #number of categories (spam or ham)                
  n_h = 50                #dimension of GloVe vectors
    
  W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
  b = np.zeros((n_y,))
  
  #convert Y to one hot with 2 classes
  Y_oh = convert_to_one_hot(Y, n_y)
    
  for t in range(num_iterations): 
    for i in range(m):  
        avg = sentence_to_avg(X[i], word_to_vec_map)
        
        #forward propagation
        z = np.dot(W, avg) + b
        a = softmax(z)
        
        #compute cost
        cost = -np.squeeze(np.sum(Y_oh[i] * np.log(a)))
        
        #compute gradients
        dz = a - Y[i]
        dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
        db = dz
        
        ## Update parameters with Stochastic Gradient Descent
        W = W - learning_rate * dW
        b = b - learning_rate * db
            
    if t % 100 == 0:
        print("Epoch: " + str(t) + " --- cost = " + str(cost))
        pred = predict(X, Y, W, b, word_to_vec_map)

  return pred, W, b

In [118]:
pred, W, b = model(x_train, y_train, word_to_vec_map)
print(pred)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':


Epoch: 0 --- cost = nan
Accuracy: 0.8660533991474085
Epoch: 100 --- cost = nan
Accuracy: 0.8660533991474085
Epoch: 200 --- cost = nan
Accuracy: 0.8660533991474085
Epoch: 300 --- cost = nan
Accuracy: 0.8660533991474085
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [0]:
print("Training set:")
pred_train = predict(x_train, y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(x_test, y_test, W, b, word_to_vec_map)

Training set:
Accuracy: 0.8660533991474085
Test set:
Accuracy: 0.8654708520179372
